In [35]:

import torch
import torch.nn as nn

ce = nn.CrossEntropyLoss(label_smoothing=0.1)
 
logits = torch.tensor([[-1.90, -0.29, -2.30], [-0.29, -1.90, -2.30]])
target = torch.tensor([[0., 1., 0.], [1., 0., 0.]])
print("Cross entropy: %.3f" % ce(logits, target))
loss =  ce(logits, target)
ce(logits, target)
loss

Cross entropy: 0.409


TypeError: 'float' object is not callable

In [102]:
class LabelSmoothingLoss(torch.nn.Module):
    def __init__(self, num_classes, smoothing=0.1):
        super(LabelSmoothingLoss, self).__init__()
        self.num_classes = num_classes
        self.max_smoothing = smoothing

    def forward(self, output, target):
 # Convert target labels to one-hot encoding
        target_one_hot = torch.zeros_like(output)
        target_one_hot.scatter_(1, target.unsqueeze(1), 1)
        print(target_smoothed)
        # Compute cross-entropy loss
        loss = -(torch.nn.functional.log_softmax(output, dim=1) * target_smoothed).sum(dim=1).mean()
        return loss

# Example usage
num_classes = 5
smoothing = 0.3
criterion = LabelSmoothingLoss(num_classes, smoothing)

# Example true labels
true_labels = torch.tensor([2, 4, 0])

# Example predicted probabilities
pred_probs = torch.tensor([[0.1, 0.2, 0.6, 0.05, 0.05],
                           [0.3, 0.1, 0.1, 0.2, 0.3],
                           [0.05, 0.05, 0.1, 0.4, 0.4]])

loss = criterion(pred_probs, true_labels)
print("Label Smoothing Loss:", loss.item())

tensor([[-3.1513e-03, -1.2605e-02,  1.0000e+00, -7.8782e-04, -7.8782e-04],
        [-3.1653e-02, -3.5170e-03, -3.5170e-03, -1.4068e-02,  1.0000e+00],
        [ 1.0000e+00, -1.0285e-03, -4.1138e-03, -6.5821e-02, -6.5821e-02]])
Label Smoothing Loss: 1.4038673639297485


In [95]:

import torch
import torch.nn as nn
print(torch.__version__)
ce = nn.CrossEntropyLoss(label_smoothing=0.2)
loss = LabelSmoothingLoss(3, 0.2)

logits = torch.tensor([[-1.90, -0.29, -2.30], [-0.29, -1.90, -2.30]])
target = torch.tensor([[1.0, 0., 0.], [0., 1., 0.]])
target2 = torch.tensor([0, 1])

print("Cross entropy built in: %.4f" % ce(logits, target2))
print( loss(logits, target2))

print("Cross entropy: %.4f" % ce(logits, target2))

print(loss(logits, target2))
ce = nn.CrossEntropyLoss()

print("Cross entropy: %.4f" % ce(logits, target))

2.2.1
Cross entropy built in: 1.8174
tensor([[0.8667, 0.0667, 0.0667],
        [0.0667, 0.8667, 0.0667]])
tensor(1.8174)
Cross entropy: 1.8174
tensor([[0.8667, 0.0667, 0.0667],
        [0.0667, 0.8667, 0.0667]])
tensor(1.8174)
Cross entropy: 1.8981


In [11]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)

print(input)
print(target)
output = ce(input, target)
print(output)

t1 = torch.tensor([[ 0.5161,  0.6198, -1.4475,  0.4004, -0.3914],
        [ 0.9718,  0.4976,  0.6293, -1.1079, -0.3110],
        [ 1.2641,  0.9328, -1.8087,  0.1155, -0.3138]], requires_grad=True)
t2 = torch.tensor([[0.5983, 0.0077, 0.3152, 0.0430, 0.0358],
        [0.2199, 0.2125, 0.4064, 0.0695, 0.0919],
        [0.4249, 0.1048, 0.3183, 0.0697, 0.0824]])

output = ce(t1, t2)
print(output)
t2single = torch.tensor([0, 2, 0])
output = ce(t1, t2single)
print(output)



tensor([[ 1.2294, -0.7213, -1.1914,  0.1214, -1.2865],
        [ 0.5542, -1.3177, -0.3021,  1.6507, -1.0291],
        [-0.4412, -0.5488, -0.7729, -1.1762,  1.2069]], requires_grad=True)
tensor([[0.0920, 0.2227, 0.2135, 0.2961, 0.1756],
        [0.0368, 0.0719, 0.5036, 0.1418, 0.2459],
        [0.2943, 0.1632, 0.2078, 0.2913, 0.0435]])
tensor(2.3083, grad_fn=<DivBackward1>)
tensor(1.8266, grad_fn=<DivBackward1>)
tensor(1.1471, grad_fn=<NllLossBackward0>)


In [31]:
# Taken from Ordinal Classification Paper provided
import math 

def soft_encoded_vector(label, penalty):
    """
    Function to encode Label into Soft Encoded Vector
    inputs:
    label (int): Ground Truth Label//.lkl;'
    
     
      
        (float): Penalty for Labels further from Ground Truth
    
    outputs:
    soft_vector (list of Float): Soft Encoded Vector of Ground Truth
    
    """
    classes = [0,1,2]
    
    cost = lambda val, label : penalty*(abs(label-val))
    expr = lambda val, label : math.exp(-cost(val, label))
    
    total_sum = sum([expr(val,label) for val in classes])
    
    soft_vector = [(expr(val,label)/total_sum) for val in classes]
    
    return (soft_vector)


def convert_labels_to_vec(labels,penalty):
    """
    Function to take list of Labels & convert all to list of soft encodings
    inputs:
    labels (list of Int): List of Labels for batch
    penalty (float): Penalty for Labels further from Ground Truth
    
    outputs:
    2d Array (list of list of Float): List of Soft Encoded Vectors
    
    """
    outputs = []
    
    for label in labels:
        outputs.append(soft_encoded_vector(label,penalty))
    
    return torch.Tensor(outputs)

# Example of Usage + Output
example_labels = torch.Tensor([2])
print(convert_labels_to_vec(example_labels,1))

tensor([[0.0900, 0.2447, 0.6652]])


In [ ]:
new_labels = original_labels * (1 - label_smoothing) + label_smoothing / num_classes